In [1]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col, sum, countDistinct

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

employees = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "employees") \
    .option("user", "root") \
    .option("password", "password") \
    .load()


performance_reviews = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "performance_reviews") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

25/09/06 19:20:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/06 19:20:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
employees.show()

+-----------+-------------+
|employee_id|         name|
+-----------+-------------+
|          1|Alice Johnson|
|          2|    Bob Smith|
|          3|  Carol Davis|
|          4| David Wilson|
|          5|   Emma Brown|
+-----------+-------------+



In [4]:
performance_reviews.show()

+---------+-----------+-----------+------+
|review_id|employee_id|review_date|rating|
+---------+-----------+-----------+------+
|        1|          1| 2023-01-15|     2|
|        2|          1| 2023-04-15|     3|
|        3|          1| 2023-07-15|     4|
|        4|          1| 2023-10-15|     5|
|        5|          2| 2023-02-01|     3|
|        6|          2| 2023-05-01|     2|
|        7|          2| 2023-08-01|     4|
|        8|          2| 2023-11-01|     5|
|        9|          3| 2023-03-10|     1|
|       10|          3| 2023-06-10|     2|
|       11|          3| 2023-09-10|     3|
|       12|          3| 2023-12-10|     4|
|       13|          4| 2023-01-20|     4|
|       14|          4| 2023-04-20|     4|
|       15|          4| 2023-07-20|     4|
|       16|          5| 2023-02-15|     3|
|       17|          5| 2023-05-15|     2|
+---------+-----------+-----------+------+



In [25]:
from pyspark.sql.functions import count, dense_rank,lead
from pyspark.sql.window import Window

In [34]:
more_than_3reviews_df = performance_reviews \
    .groupBy(col("employee_id")) \
    .agg(count("review_id").alias("review_cnt")) \
    .filter(col("review_cnt") >= 3) \
    .withColumnRenamed("employee_id", "id")

employee_performance_df = more_than_3reviews_df \
    .join(performance_reviews, on= more_than_3reviews_df.id == performance_reviews.employee_id, how="inner") \
    .select("employee_id", "review_date", "rating") \
    .withColumn("drnk", dense_rank().over(Window.partitionBy("employee_id").orderBy(col("review_date").desc()))) \
    .filter(col("drnk") <= 3) \
    .withColumn("next_rating",
                lead(col("rating")).over(Window.partitionBy("employee_id").orderBy(col("review_date").desc()))) \
    .withColumn("next_follow_rating",
                lead(col("rating"), 2).over(Window.partitionBy("employee_id").orderBy(col("review_date").desc()))) \
    .filter(
    (col("next_follow_rating").isNotNull())
    & (col("rating") > col("next_rating"))
    & (col("next_rating") > col("next_follow_rating"))
) \
    .withColumn("improvement_score", col("rating") - col("next_follow_rating")) \
    .select("employee_id", "improvement_score") \
employees = employees.withColumnRenamed("employee_id", "emp_id")

result_df = employee_performance_df \
    .join(employees, on=employee_performance_df.employee_id == employees.emp_id, how="inner") \
    .select("employee_id", "name", "improvement_score") \
    .orderBy(col("improvement_score").desc(), col("name"))


+-----------+-------------+-----------------+
|employee_id|         name|improvement_score|
+-----------+-------------+-----------------+
|          2|    Bob Smith|                3|
|          1|Alice Johnson|                2|
|          3|  Carol Davis|                2|
+-----------+-------------+-----------------+

